In [1]:
import numpy as np

def stride_attention_matrix(query, key, stride_tokens):
    """
    Calcola la Stride Attention Matrix tra query e key utilizzando i token specificati come stride.

    Argomenti:
    query: array numpy con forma (num_queries, embedding_dim)
    key: array numpy con forma (num_keys, embedding_dim)
    stride_tokens: array di indici dei token con cui calcolare l'attenzione

    Ritorna:
    stride_attention: array numpy con forma (num_queries, len(stride_tokens))
    """

    num_queries, embedding_dim = query.shape
    num_stride_tokens = len(stride_tokens)

    # Estrai i token dalla chiave utilizzando gli indici specificati
    stride_key = key[stride_tokens]

    # Calcolo dei prodotti tra query e stride_key
    scores = np.matmul(query, stride_key.transpose())  # (num_queries, len(stride_tokens))

    return scores

# Esempio di utilizzo
query = np.array([[1, 2, 3],
                  [4, 5, 6]])
key = np.array([[0, 1, 2],
                [3, 4, 5],
                [6, 7, 8],
                [9, 10, 11]])

# Indici dei token con cui calcolare l'attenzione
stride_tokens = [0, 2]

result = stride_attention_matrix(query, key, stride_tokens)
print("Stride Attention Matrix:")
print(result)


Stride Attention Matrix:
[[  8  44]
 [ 17 107]]


In [12]:
import numpy as np

def stride_attention_matrix(query, key, stride_tokens):
    """
    Calcola la Stride Attention Matrix tra query e key utilizzando i token specificati come stride.

    Argomenti:
    query: array numpy con forma (num_queries, embedding_dim)
    key: array numpy con forma (num_keys, embedding_dim)
    stride_tokens: array di indici dei token con cui calcolare l'attenzione

    Ritorna:
    stride_attention: array numpy con forma (num_queries, len(stride_tokens))
    """

    num_queries, embedding_dim = query.shape
    num_stride_tokens = len(stride_tokens)

    stride_attention = np.zeros((num_queries, num_stride_tokens))

    for i, token_index in enumerate(stride_tokens):
        # Estrai il token corrente dalla chiave
        stride_key = key[token_index]

        # Calcola l'attenzione solo tra la query corrispondente e il token corrente
        scores = np.matmul(query, stride_key)

        stride_attention[:, i] = scores.squeeze()

    return stride_attention

# Esempio di utilizzo
query = np.array([[1, 2, 3]])
key = np.array([[0, 1, 2],
                [3, 4, 5],
                [6, 7, 8],
                [9, 10, 11]])

# Indici dei token con cui calcolare l'attenzione
stride_tokens = [0, 2]

result = stride_attention_matrix(query, key, stride_tokens)
print("Stride Attention Matrix:")
print(result)


Stride Attention Matrix:
[[ 8. 44.]]


In [14]:
import numpy as np

def stride_attention_matrix(query, key, query_index, stride_tokens):
    """
    Calcola la Stride Attention Matrix tra una specifica query e le relative key utilizzando i token specificati come stride.

    Argomenti:
    query: array numpy con forma (num_queries, embedding_dim)
    key: array numpy con forma (num_keys, embedding_dim)
    query_index: indice della query di interesse
    stride_tokens: array di indici dei token con cui calcolare l'attenzione

    Ritorna:
    stride_attention: array numpy con forma (1, len(stride_tokens))
    """

    embedding_dim = query.shape[1]
    num_stride_tokens = len(stride_tokens)

    stride_attention = np.zeros((1, num_stride_tokens))

    # Estrai la query di interesse
    selected_query = query[query_index]

    for i, token_index in enumerate(stride_tokens):
        # Estrai il token corrente dalla chiave
        stride_key = key[token_index]

        # Calcola l'attenzione solo tra la query corrispondente e il token corrente
        score = np.dot(selected_query, stride_key)

        stride_attention[:, i] = score

    return stride_attention

# Esempio di utilizzo
queries = np.array([[1, 2, 3],
                    [4, 5, 6],
                    [7, 8, 9],
                    [10, 11,12]])

keys = np.array([[0, 1, 2],
                 [3, 4, 5],
                 [6, 7, 8],
                 [9, 10, 11]])

# Indice della query di interesse
query_index = 1

# Indici dei token con cui calcolare l'attenzione
stride_tokens = [0, 2]

result = stride_attention_matrix(queries, keys, query_index, stride_tokens)
print("Stride Attention Matrix:")
print(result)


Stride Attention Matrix:
[[ 17. 107.]]


In [4]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
import numpy as np
import matplotlib.pyplot as plt

# Definizione della stride_attention_matrix
def stride_attention_matrix(query, key, stride_tokens):
    """
    Calcola la Stride Attention Matrix tra query e key utilizzando i token specificati come stride.

    Argomenti:
    query: array numpy con forma (num_queries, embedding_dim)
    key: array numpy con forma (num_keys, embedding_dim)
    stride_tokens: array di indici dei token con cui calcolare l'attenzione

    Ritorna:
    stride_attention: array numpy con forma (num_queries, len(stride_tokens))
    """

    num_queries, embedding_dim = query.shape
    num_stride_tokens = len(stride_tokens)

    # Estrai i token dalla chiave utilizzando gli indici specificati
    stride_key = key[stride_tokens]

    # Calcolo dei prodotti tra query e stride_key
    scores = np.matmul(query, stride_key.transpose())  # (num_queries, len(stride_tokens))

    return scores

# Carica il tokenizer di BERT e il modello preaddestrato
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Testo di input
text = "The capital of France is Paris"

# Tokenizza il testo e converte in tensori
tokens = tokenizer(text, return_tensors='pt')
attention_mask = tokens['attention_mask']

# Applica la maschera di attenzione per mascherare tutto tranne 'capital'
capital_index = tokens['input_ids'].tolist()[0].index(tokenizer.convert_tokens_to_ids('capital'))
attention_mask[:, :capital_index] = 0
attention_mask[:, capital_index+1:] = 0

# Esegui l'output del modello
outputs = model(input_ids=tokens['input_ids'], attention_mask=attention_mask)

# Recupero l'embedding per i token
with torch.no_grad():
    model_output = model.base_model(input_ids=tokens['input_ids'], attention_mask=attention_mask)
embedding_output = model_output.last_hidden_state

# Estrai l'embedding per i token del testo
embeddings = embedding_output[0]  # prendi solo il primo elemento (non stiamo elaborando un batch)

# Definizione degli stride per calcolare l'attenzione tra i token specifici
strides = {
    'capital': [3, 4],  # 'is' e 'France'
    'France': [1],      # 'capital'
}

for token, stride_tokens in strides.items():
    # Verifica se il token è presente nell'input
    if token in tokenizer.vocab:
        # Indice del token nell'input
        token_index = tokens['input_ids'].tolist()[0].index(tokenizer.convert_tokens_to_ids(token))

        # Estrai l'embedding del token
        query = embeddings[token_index].unsqueeze(0)

        # Estrai l'embedding dei token specificati dagli stride
        key_indices = [tokens['input_ids'].tolist()[0].index(tokenizer.convert_tokens_to_ids(token)) for token in tokenizer.convert_ids_to_tokens(tokens['input_ids'].tolist()[0]) if tokenizer.convert_tokens_to_ids(token) in stride_tokens]

        # Verifica se ci sono indici validi
        if len(key_indices) > 0:
            key = embeddings[key_indices]

            # Calcola la Stride Attention Matrix solo se ci sono token da considerare
            stride_attention = stride_attention_matrix(query.numpy(), key.numpy(), list(range(len(stride_tokens))))

            print(f"Attività dell'attenzione per l'embedding '{token}':")
            print(stride_attention)

            # Creazione della matrice di attenzione sparsa
            attention_matrix = np.zeros((len(stride_tokens), len(stride_tokens)))
            for i, token_i in enumerate(stride_tokens):
                for j, token_j in enumerate(stride_tokens):
                    attention_matrix[i, j] = stride_attention[0, j]

            # Plot della matrice di attenzione sparsa
            plt.imshow(attention_matrix, cmap='Blues', interpolation='nearest')
            plt.title(f'Attention Matrix for token "{token}"')
            plt.xlabel('Token Indices')
            plt.ylabel('Token Indices')
            plt.colorbar()
            plt.show()
        else:
            print(f"Nessun token trovato per il calcolo dell'attenzione per l'embedding '{token}'.")
    else:
        print(f"Il token '{token}' non è presente nel vocabolario del tokenizer.")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Nessun token trovato per il calcolo dell'attenzione per l'embedding 'capital'.
Il token 'France' non è presente nel vocabolario del tokenizer.


In [5]:
# Testo di input
text = "The capital of France is Paris"

# Tokenizzazione del testo
tokens = text.split()

# Definizione degli indici dei token per cui calcolare l'attenzione
stride_tokens = [1, 4]  # "capital" e "Paris"

# Creazione della matrice one-hot encoding
one_hot_matrix = np.eye(len(tokens))

# Rappresentazione one-hot del testo
query = one_hot_matrix[[1, 4]]  # Selezioniamo le righe corrispondenti ai token "capital" e "Paris"

# Rappresentazione one-hot dei token della chiave
key = one_hot_matrix[:len(tokens)]

# Calcolo della Stride Attention Matrix
result = stride_attention_matrix(query, key, stride_tokens)

print("Stride Attention Matrix:")
print(result)


Stride Attention Matrix:
[[1. 0.]
 [0. 1.]]


In questo esempio, le parole della frase sono state rappresentate come vettori di embedding e la funzione stride_attention_matrix viene utilizzata per calcolare la matrice di attenzione tra le query (le parole della frase) e la chiave (che sono vettori casuali generati in questo esempio). Si considerano solo il primo e il terzo token della chiave per calcolare l'attenzione.

In [10]:

import numpy as np

# Definizione degli embedding per le parole della frase
word_embeddings = {
    'Oggi': [0.1, 0.2, 0.3],
    'il': [0.2, 0.3, 0.4],
    'sole': [0.3, 0.4, 0.5],
    'splende': [0.4, 0.5, 0.6],
    'ed': [0.5, 0.6, 0.7],
    'è': [0.6, 0.7, 0.8],
    'una': [0.7, 0.8, 0.9],
    "c'è": [0.8, 0.9, 1.0],
    'pure': [0.9, 1.0, 1.1],
    'caldo': [1.0, 1.1, 1.2]
}

# Frase di esempio
sentence = 'Oggi il sole splende ed è una c\'è pure caldo'

# Convertire la frase in una lista di parole
words = sentence.split()

# Ottenere gli embedding per ogni parola della frase
word_vectors = [word_embeddings[word] for word in words]

# Convertire la lista di embedding in un array numpy
query = np.array(word_vectors)

# Key di esempio (creati casualmente)
key = np.random.rand(10, 3)

# Indici dei token con cui calcolare l'attenzione
stride_tokens = [0, 2]  # Ad esempio, consideriamo solo il primo e il terzo token della key
query_token = [3]

# Funzione per calcolare la Stride Attention Matrix
def stride_attention_matrix(query, key, stride_tokens):
    #num_queries, embedding_dim = query.shape
    stride_key = key[stride_tokens]
    query
    scores = np.matmul(query, stride_key.transpose())  
    return scores

# Calcolo della Stride Attention Matrix
result = stride_attention_matrix(query, key, stride_tokens)

# Stampare il risultato
print("Stride Attention Matrix:")
print(result)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 10)